### Streams
Java Streams API lets us manipulate collections of data in a declarative way. One effect is that the code size can be significantly reduced as the below example shows:

In [5]:
int[] numbers = {5, 7, 2, 9, 8, 1, 7};

// Copy the above array since we don't want to mutate it
int[] result = Arrays.copyOf(numbers, numbers.length);

// Sort it
Arrays.sort(result);

// Get sum of lowest 3
int sum = 0;
for(int i=0; i<3; i++){
    sum += result[i];
}

System.out.println(sum);

8


The same operation can be written in a concise manner using Streams

In [4]:
int[] numbers = {5, 7, 2, 9, 8, 1, 7};

int sum = Arrays.stream(numbers)
                .sorted()
                .limit(3)
                .sum();

System.out.println(sum);

8


### Building Streams

Object streams are the general kind of stream. To create an object stream:

In [ ]:
// From values
Stream<String> cityStream = Stream.of("Los Angeles", "Paris", "Tokyo", "Berlin");

// From arrays
String[] cities = new String[] { "Los Angeles", "Paris", "Tokyo", "Berlin" };
Stream<String> sameCityStream = Arrays.stream(cities);

// From collections
List<String> cityList = Arrays.asList(cities);
Stream<String> anotherCityStream = cityList.stream();

We have specialised Streams too: `IntStream`, `LongStream` and `DoubleStream` .  
![Stream Inheritance](https://i.stack.imgur.com/uI6XA.png)  

The stream method of Arrays is smart enough to return these types of stream

In [ ]:
int[] primes = new int[] { 2, 3, 5, 7, 11 };
IntStream primeStream = Arrays.stream(primes);

// list's stream method returns object streams

Or use the above specialised stream's static methods

In [ ]:
DoubleStream doubleSteam = DoubleStream.of(3.56, 2.91, 8.314);

LongStream longStream = LongStream.range(1, 101);

We can also convert an object stream to specialised stream

In [ ]:
Stream<Integer> numbers = Stream.of(1, 4, 7, 8, 0, -5);
IntStream integerNumbers = numbers.mapToInt(i -> i);

### Intermediate Operations
Intermediate operations on stream return stream. These operations can be chained together and are lazy evaluated (when a terminal operation is called).

| Operation 	| Description 	|
|-	|-	|
| sorted() 	| Returns a stream consisting of the elements of this stream, sorted according to natural order. 	|
| skip(long n) 	| Returns a stream consisting of the remaining elements of this stream after discarding the first n elements of the stream. 	|
| peek(Consumer<? super T> action) 	| Returns a stream consisting of the elements of this stream, additionally performing the provided action on each element as elements are consumed from the resulting stream. 	|
| limit(long maxSize) 	| Returns a stream consisting of the elements of this stream, truncated to be no longer than maxSize in length. 	|
| distinct() 	| Returns a stream consisting of the distinct elements (according to Object.equals(Object)) of this stream. 	|
| filter(Predicate<? super T> predicate) 	| Returns a stream consisting of the elements of this stream that match the given predicate. 	|
| map(Function<? super T,? extends R> mapper) 	| Returns a stream consisting of the results of applying the given function to the elements of this stream. 	|
| flatMap(Function<? super T,? extends Stream<? extends R>> mapper) 	| Returns a stream consisting of the results of replacing each element of this stream with the contents of a mapped stream produced by applying the provided mapping function to each element. 	|

The lazy evalutaion characteristic can be seen in the below example:

In [ ]:
// Nothing printed
Stream.of("d2", "a2", "b1", "b3", "c")
    .filter(s -> {
        System.out.println("filter: " + s);
        return true;
    });

Some operations can also be combined together while evaluating all the chained operations.

### Terminal Operations
These operations typically return a single value. Some examples:

In [9]:
// Matching terminal operations
List<String> movies = List.of("One flew over the cuckoo's nest", "To kill a mockingbird", "Gone with the wind");

// Does any element in stream match the condition?
System.out.println(movies.stream().anyMatch(s -> s.startsWith("To")));

// Do all elements in stream match the condition?
System.out.println(movies.stream().allMatch(s -> s.startsWith("One")));

// Do none of the elements in stream match the condition?
System.out.println(movies.stream().noneMatch(s -> s.startsWith("Once")));

true
false
true


In [15]:
// Minimum, maximum, count
List<Integer> integers = List.of(12, 22, 45, 65, 5, 87);

// max() : Provide a Comparator, returns an Optional, since stream could have
// been empty
integers.stream().max((a, b) -> a - b).ifPresent(System.out::println);

// Counting
System.out.println(integers.stream().count());

87
6


In [19]:
// Reduce

// Sum all the elements
integers.stream().reduce((a, b) -> a + b).ifPresent(System.out::println);

// Using below class for examples
class Person {
    String name;
    int age;

    Person(String name, int age) {
        this.name = name;
        this.age = age;
    }

    @Override
    public String toString() {
        return name;
    }
}

List<Person> persons = Arrays.asList(new Person("Max", 18), new Person("Peter", 23), new Person("Pamela", 23), new Person("David", 12));

// Person with maximum age
persons.stream().reduce((p1, p2) -> p1.age > p2.age ? p1 : p2).ifPresent(System.out::println);

// Reduce may also accept an identity element
// in that case, it doesn't return Optional
System.out.println(persons.stream().reduce(new Person("", 0), (p1, p2) -> { p1.age += p2.age; p1.name += p2.name; return p1;}));

236
Pamela
MaxPeterPamelaDavid


`IntStream`, `LongStream`, `DoubleStream` provide some other terminal operations like `sum`, `average`, `summaryStatistics`.

The `collect` terminal operator is one of the most important terminal operator. With collect operation, we can use the result to form a collection, map, set, etc. We make use of the many static methods of the `Collectors` class. This class also contains methods to get maximum, minimum, join together strings, etc.

In [ ]:
// Collect to a list
List<Integer> ageList = persons.stream().map(p -> p.age).sorted().collect(Collectors.toList());

// Join together as string
String s = persons.stream().map(p -> p.name).collect(Collectors.joining(", "));
// The same can be achieved using reduce, but is a bit clunky
String[] s_ = { "" };
persons.stream().map(p -> p.name).reduce((a, b) -> a + ", " + b).ifPresent(a -> s_[0] = a);

// Group by, collect to map
Map<Integer, List<Person>> ageToPersonMap = persons.stream().collect(Collectors.groupingBy((p) -> p.age));
// What if we want age-> name mapping?
Map<Integer, String> ageToNameMap = persons.stream().collect(Collectors.toMap(p -> p.age, p -> p.name, (n1, n2) -> n1 + "," + n2));

### Optional
Consider the classes below:

In [ ]:
public class Person {
    private Car car;
    public Car getCar() { return car; }
}

public class Car {
    private Insurance insurance;
    public Insurance getInsurance() { return insurance; }
}

public class Insurance {
    private String name;
    public String getName() { return name; }
}

Now if a person does not have a car, calling `p.getCar()` would return null. Acting upon a null reference can lead to program exceptions. A solution to this is to introduce defensive null checking

In [ ]:
// Too much nesting
public string getCarInsuranceName(Person p){
    if(p != null){
        Car c = p.getCar();
        if(c != null){
            Insurance i = c.getInsurance();
            if(i != null){
                return i.getName();
            }
        }
    }
    
    return 'Unknown';
}

Another way would be:

In [ ]:
// Too many repetitions
public string getCarInsuranceName(Person p){
    if(p == null){
        return 'Unknown';
    }
    
    Car c = p.getCar();
    if(c == null){
        return 'Unknown';
    }
    
    Insurance i = c.getInsurance();
    if(i == null){
        return 'Unknown';
    }
    
    return i.getName();
}

Using `Optional`, we can model our previous classes like:

In [ ]:
public class Person {
    private Optional<Car> car;
    public Optional<Car> getCar() { return car; }
}

public class Car {
    private Optional<Insurance> insurance;
    public Optional<Insurance> getInsurance() { return insurance; }
}

public class Insurance {
    private String name;
    public String getName() { return name; }
}

The use of Optional enriches the semantics of your model. The fact that a person references an `Optional<Car>`, and a car an `Optional<Insurance>`, makes it explicit in the domain that a person might or might not own a car, and that car might or might not be insured. The name of insurance is not optional, thereby signalling that it is a mandatory field.

In [ ]:
// Presence of no car
Optional<Car> noCar = Optional.empty();

// Presence of car
Optional<Car> car = Optional.of(new Car());

To get the car from an optional,

In [ ]:
Car c = car.get()

But, the get method return `NoSuchElementException`, for which we would have to add try-catch block. So this brings us back to where we started!